In [4]:
#### IMPORT LIBS ####
import subprocess, os, glob
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logomaker
from Bio.Seq import Seq
from Bio import SeqIO
from Bio import motifs

#### DEFINE FUNCTIONS ####
## MISC: Data processing
def rev_comp(seq):
    five_prime = Seq(seq)
    rc = str(five_prime.reverse_complement())
    return rc

def write_df_to_fasta(df, title, col="Kmer"):
    for_fasta = zip(np.arange(0, len(list(df[col]))), list(df[col]))
    with open(f"{title}.fa", "w") as f:
        for seq in for_fasta:
            f.write(f">{seq[0]}\n")
            f.write(f"{seq[1]}\n")
    return None

def get_fasta_seqs(fasta_file):
    sequences = []
    # Open and parse the FASTA file
    with open(fasta_file, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            sequences.append(record.seq)
            
    return sequences

def fastp_merge_reads(r1, r2, base):
    merged_file = f"{base}_merged_reads.fastq.gz"
    fastp_str = (
        f"fastp --in1={r1} --in2={r2} --merge --out1={base}_trimmed_R1.fastq.gz --out2={base}_trimmed_R2.fastq.gz "
        f"--merged_out={merged_file} --html={base}_report.html --json={base}report.json"
    )
    print(fastp_str)
    os.system(fastp_str)
    return merged_file

def grep_fastq(fasta, left_pat, right_pat, length_variable_region, out_file_name):
    """
    INPUTS:
        - fasta: REQUIRED. gzip or uncompressed accepted
        - left_pat: REQUIRED. 5'/left-flanking sequence for variable region
        - right_pat: REQUIRED. 3'/right-flanking sequence for variable region
        - length_variable_region: REQUIRED. EXACT number of N's comprising variable region
                    between the flanking sequences
        - out_file_name: REQUIRED. Name for output file containing matches
    OUTPUTS:
        - out_file_name: Name of file saved to store matches
        - sequencing_depth: Number of total reads in fasta file
        - number_of_pattern_matches: Total number of grep hits (NOT unique number of patterns)
    """
    # IF GZIP
    if fasta.endswith(".gz"):
        # GET SEQUENCING DEPTH
        sequencing_depth = int(subprocess.check_output(
            f"zcat {fasta}|wc -l", shell=True).decode('utf-8')) / 2

        # GET MATCHES
        grep_str = (
            f"zcat {fasta}| grep -Eo "
            f"'{left_pat}[A|T|G|C]{{{length_variable_region}}}{right_pat}' "
            f"| sed -e 's/{right_pat}$//' | sed -e 's/^{left_pat}//' > {out_file_name}"
        )
        print(grep_str)
        subprocess.run(grep_str, shell=True)

        # COUNT NUMBER OF MATCHES
        number_of_pattern_matches = int(subprocess.check_output(
            f"wc -l {out_file_name}", shell=True).decode('utf-8').split()[0])
    # ELIF NOT COMPRESSED:
    else:
        # GET SEQUENCING DEPTH
        sequencing_depth = int(subprocess.check_output(
            f"wc -l {fasta}", shell=True).decode('utf-8').split()[0]) / 2

        # GET MATCHES
        grep_str = (
            f"grep -Eo '{left_pat}[A|T|G|C]{{{length_variable_region}}}{right_pat}' "
            f"{fasta} | sed -e 's/{right_pat}$//' | sed -e 's/^{left_pat}//' > {out_file_name}"
        )
        print(grep_str)
        subprocess.run(grep_str, shell=True)

        # COUNT NUMBER OF MATCHES
        number_of_pattern_matches = int(subprocess.check_output(
            f"wc -l {out_file_name}", shell=True).decode('utf-8').split()[0])

    return out_file_name, sequencing_depth, number_of_pattern_matches


# KMER FREQUENCY
def get_kmer_frequency_dict(pam_match_file, sequencing_depth=None, reverse_complement=False,
                            bc_trim_n_left=False, bc_trim_n_right=False):
    """
    INPUT:
        - pam_match_file: REQUIRED. Fasta or txt file of PAM pattern-matches. NOT FASTQ compatible.
        - sequencing_depth: OPTIONAL. Default = None. Sequencing depth (int) for optional normalization.
        - reverse_complement: OPTIONAL. Default = False. Option to reverse complement the PAM/grep output.
        - bc_trim_n_left: OPTIONAL. Default = False. Option to trim N number of bases from 5'-end of PAM/grep output.
                            Trim end as it relates to the PAM-match file, not reverse compliment.
        - bc_trim_n_right: OPTIONAL. Default = False. Option to trim N number of bases from 3'-end of PAM/grep output.
                            Trim end as it relates to the PAM-match file, not reverse compliment.

    OUTPUT:
        - kmer_freq_dict: Dictionary of kmer frequencies. Keys are the Kmer sequences.
    """
    with open(pam_match_file, "r") as f:
        pam_seqs = [x.replace("\n", "") for x in f.readlines() if not x.startswith(">")]

    ## TRIM
    if bc_trim_n_left:
        pam_seqs = [x[bc_trim_n_left:] for x in pam_seqs]
    if bc_trim_n_right:
        pam_seqs = [x[:-1 * bc_trim_n_right] for x in pam_seqs]

    ## COUNT
    if sequencing_depth:
        kmer_freq_dict = {k: v / sequencing_depth for k, v in Counter(pam_seqs).items()}
    else:
        kmer_freq_dict = dict(Counter(pam_seqs))

    ## REVERSE COMPLEMENT
    if reverse_complement:
        kmer_freq_dict = {rev_comp(k): v for k, v in kmer_freq_dict.items()}

    return kmer_freq_dict

# NUCLEOTIDE FREQUENCY
def get_nucleotide_frequency_dict(pam_match_file, length_variable_region=8, sequencing_depth=None,
                                  reverse_complement=False, bc_trim_n_left=False, bc_trim_n_right=False):
    """
    INPUT:
        - pam_match_file: REQUIRED. Fasta or txt file of PAM pattern-matches. NOT FASTQ compatible.
        - length_variable_region: REQUIRED. Default = 8. EXACT number of N's comprising variable region
                            between the flanking sequences
        - sequencing_depth: OPTIONAL. Default = None. Sequencing depth (int) for optional normalization.
        - reverse_complement: OPTIONAL. Default = False. Option to reverse complement the PAM/grep output.
        - bc_trim_n_left: OPTIONAL. Default = False. Option to trim N number of bases from 5'-end of PAM/grep output.
                            Trim end as it relates to the PAM-match file, not reverse compliment.
        - bc_trim_n_right: OPTIONAL. Default = False. Option to trim N number of bases from 3'-end of PAM/grep output.
                            Trim end as it relates to the PAM-match file, not reverse compliment.

    OUTPUT:
        - kmer_freq_dict: Dictionary of kmer frequencies. Keys are the Kmer sequences.
    """

    nucleotide_frequency_dict = {}
    nucleotide_order = ["A", "C", "G", "T"]

    ## READ PAM FILE
    with open(pam_match_file) as f:
        pam_seqs = [x.replace("\n", "") for x in f.readlines() if not x.startswith(">")]

    ## TRIM
    if bc_trim_n_left:
        pam_seqs = [x[bc_trim_n_left:] for x in pam_seqs]
    if bc_trim_n_right:
        pam_seqs = [x[:-1 * bc_trim_n_right] for x in pam_seqs]

    ## REVERSE COMPLEMENT
    if reverse_complement:  # Reverse complement sequence
        pam_seqs = [rev_comp(x) for x in pam_seqs]

    ## CALCULATE NUCLEOTIDE FREQUENCY AT EACH POSITION IN PAM
    # GET PAM LENGTH FOR CALCULATION
    if bc_trim_n_left or bc_trim_n_right:
        pam_length = length_variable_region - bc_trim_n_left - bc_trim_n_right
    else:
        pam_length = length_variable_region

    for i in range(pam_length):
        if sequencing_depth:
            temp_dict = {k: v / sequencing_depth for k, v in Counter([x[i] for x in pam_seqs]).items()}
        else:
            temp_dict = {k: v for k, v in Counter([x[i] for x in pam_seqs]).items()}

        # ADD UNOBSERVED NUCLEOTIDES TO DICT
        missing = [x for x in nucleotide_order if x not in temp_dict.keys()]
        for m in missing:
            temp_dict[m] = 0

        nucleotide_frequency_dict[i] = temp_dict

    return nucleotide_frequency_dict

## PLOT LOGOS
def logo_from_fasta(fasta, graph_title = "Logo", pam_len = 6, x_tick_list = False, 
                    to_save = True, out_name = False):
    ## READ FASTA
    sequences = []
    for record in SeqIO.parse(fasta, "fasta"):
        sequences.append(str(record.seq))
        
    if len(sequences) == 0:
        print(f"!!!!!EMPTY FASTA:{fasta}")
        return None
    
    ## LOGOMAKER WORKFLOW
    # GET COUNT MATRIX
    counts_matrix = logomaker.alignment_to_matrix(sequences)
    # GET FREQUENCIES
    frequencies_matrix = logomaker.transform_matrix(counts_matrix, 
                                                    from_type='counts', 
                                                    to_type='probability',
                                                    #background=4*[0.25], 
                                                    pseudocount=0.0 # Avoid pseudocounting PAMs not in NegCtrl
                                                   )
    # GET INFORMATION CONTENT
    info_content_matrix = logomaker.transform_matrix(frequencies_matrix, 
                                                     from_type='probability', 
                                                     to_type='information', 
                                                     #background=4*[0.25], 
                                                     pseudocount=0.0 # Avoid pseudocounting PAMs not in NegCtrl
                                                    )

    ## PLOT LOGO
    fig, ax = plt.subplots(1, 1, figsize=[8, 6])
    logo = logomaker.Logo(info_content_matrix,
                          ax = ax, 
                          color_scheme= {"A":"orange", "T":"orange", "C":"blue", "G":"blue"}
                          )
    
    # FORMAT
    logo.ax.set_ylabel('Bits')
    logo.ax.set_ylim([0, 2])
    if x_tick_list:
        x_tick_positions = range(0,len(x_tick_list))
        x_tick_labels = x_tick_list
        logo.ax.set_xticks(x_tick_positions)
        logo.ax.set_xticklabels(x_tick_labels)
    else:
        x_tick_positions = range(0,pam_len)
        x_tick_labels = range(-1*pam_len,0)
        logo.ax.set_xticks(x_tick_positions)
        logo.ax.set_xticklabels(x_tick_labels)
    logo.ax.set_title(graph_title, size = 12)
    plt.tight_layout()
    
    # SAVE: dpi
    if to_save and out_name:
        logo.fig.savefig(out_name, format = "svg", dpi = 400)
    elif to_save and (out_name == False):
        logo.fig.savefig("Logomaker_motif.fasta", format = "svg", dpi = 400)
        
    return frequencies_matrix

def pam_fold_change_logos(neg_ctrl_pam_file, 
                          test_condition_pam_file,
                          fold_change_magnitudes=[2, 3, 5],
                          depletion=True,
                          use_pseudo_counts=False,
                          length_variable_region=8,
                          neg_ctrl_sequencing_depth=None,
                          test_condition_sequencing_depth=None,
                          reverse_complement=False,
                          bc_trim_n_left=False,
                          bc_trim_n_right=False,
                          weblogo_graph_title=False,
                          weblogo_x_axis_ticks=False,
                          out_dir=None
                          ):
    """
    DESCRIPTION:
        Plots classic weblogo (bits on y-axis), given extracted sequence matches from the targeting 
        and non-targeting conditions.
    INPUTS:
        - neg_ctrl_pam_file: REQUIRED. Fasta or txt file of negative-control PAM pattern-matches.
                            NOT FASTQ compatible.
        - test_condition_pam_file: REQUIRED. Fasta or txt file of test-condition PAM pattern-matches.
                            NOT FASTQ compatible.
        - fold_change_magnitudes: OPTIONAL. Default = [2, 3, 5]. List of log2 fold-change cutoffs for filtering
                            PAMs prior to logo generation. ENTER POSITIVE VALUES ONLY. If running a depletion
                            assay, set depletion == True in lieu of using negative values.
        - depletion: OPTIONAL. Default = True. True implies a PAM depletion assay. Set equal to False for PAM
                            enrichment assay.
        - use_pseudo_counts: OPTIONAL. Default = False. Setting pseudocounts to True, will replace any PAM with
                            count 0 with 1/number_reads_in_targeting_sample_fastq. If pseudocounts is False, 
                            then only PAM sequences found in BOTH the negative control and test condition 
                            will use for the analysis.
        - length_variable_region: OPTIONAL. Default = 8. EXACT number of N's comprising variable region
                            between the flanking sequences in the screening construct.
        - neg_ctrl_sequencing_depth: OPTIONAL. Default = None. Negative-control sequencing depth (int)
                            for optional normalization. If set to None, PAM counts will not be normalized.
        - test_condition_sequencing_depth: OPTIONAL. Default = None. Test-condition sequencing depth (int)
                            for optional normalization. If set to None, PAM counts will not be normalized.
        - reverse_complement: OPTIONAL. Default = False. Option to reverse complement the PAM/grep output.
        - bc_trim_n_left: OPTIONAL. Default = False. Option to trim N number of bases from 5'-end of 
                            PAM/grep output. Trim end as it relates to the PAM-match file, 
                            not reverse compliment.
        - bc_trim_n_right: OPTIONAL. Default = False. Option to trim N number of bases from 3'-end of 
                            PAM/grep output. Trim end as it relates to the PAM-match file, not 
                            reverse compliment.
        - weblogo_x_axis_ticks: OPTIONAL. Default = False. X-axis tick marks for weblogo.
        - save_weblogo: OPTIONAL. Default = False. Set to True, in order to save nucleotide-proportion graph.
                            Bit-Weblogos automatically save.
        - out_dir: OPTIONAL. Default = False. Directory for saving plots. Otherwise outputs to PWD.
    OUTPUTS:
        - fold_dfs: Dictionary of PAM frequencies at each specified fold-change.
        - freq_dfs: Dictionary of nucleotide frequencies at each position of PAM for each specified fold-change filter.
        - count_df: Dataframe of all PAM counts.
    """

    #### SETUP ####
    if out_dir:
        if not out_dir.endswith("/"):
            out_dir = out_dir + "/" # Enforce terminal "/"

    ## GET PAM LENGTH FOR CALCULATION
    if bc_trim_n_left or bc_trim_n_right:
        pam_length = length_variable_region - bc_trim_n_left - bc_trim_n_right
    else:
        pam_length = length_variable_region

    ## GET KMER FREQUENCIES
    ntc = get_kmer_frequency_dict(neg_ctrl_pam_file,
                                  sequencing_depth=neg_ctrl_sequencing_depth,
                                  reverse_complement=reverse_complement,
                                  bc_trim_n_left=bc_trim_n_left,
                                  bc_trim_n_right=bc_trim_n_right)
    ntc_df = pd.DataFrame({"Kmer": ntc.keys(), "Count": ntc.values()})

    test = get_kmer_frequency_dict(test_condition_pam_file,
                                   sequencing_depth=test_condition_sequencing_depth,
                                   reverse_complement=reverse_complement,
                                   bc_trim_n_left=bc_trim_n_left,
                                   bc_trim_n_right=bc_trim_n_right)
    test_df = pd.DataFrame({"Kmer": test.keys(), "Count": test.values()})

    ## OPTIONAL PSEUDO-COUNTS:
    """
        - If PAM missing in one experiment, update the NAN to 1 or 1/seq_depth_condition_with_NAN.
        - ONLY implement pseudo-counts on for the union of PAMs found test and negative-control samples, 
          NOT all 4**N combinations. This helps limit intrinisic bias for in vivo experiments and 
          substrate libraries.
    """
    if use_pseudo_counts:
        count_df = pd.merge(ntc_df, test_df, on="Kmer", suffixes=["_NTC", "_Test"], how="outer")

        ntc_cols = [x for x in count_df.columns if x.endswith("_NTC")]
        if test_condition_sequencing_depth != None:
            count_df[ntc_cols] = count_df[ntc_cols].fillna(1 / test_condition_sequencing_depth)
        else:
            count_df[ntc_cols] = count_df[ntc_cols].fillna(1)
        
        test_cols = [x for x in count_df.columns if x.endswith("_Test")]
        if test_condition_sequencing_depth != None:
            count_df[test_cols] = count_df[test_cols].fillna(1 / test_condition_sequencing_depth)
        else:
            count_df[test_cols] = count_df[test_cols].fillna(1)
    else:  # Else, don't apply pseudocounts and prevent NAN's with inner-join
        count_df = pd.merge(ntc_df, test_df, on="Kmer", suffixes=["_NTC", "_Test"], how="inner")

    ## CALCULATE FOLD-CHANGES
    count_df["Log2_Fold_Change"] = np.log2(count_df.Count_Test / count_df.Count_NTC)

    ## FILTER ON DEPLETION/ENRICHMENT CUTOFFS
    if depletion:
        fold_dfs = {f"{fc}": count_df.loc[(count_df.Log2_Fold_Change <= -1 * fc)] \
                    for fc in fold_change_magnitudes}
    else:  # Else enrichment analysis
        fold_dfs = {f"{fc}": count_df.loc[(count_df.Log2_Fold_Change >= 1 * fc)] \
                    for fc in fold_change_magnitudes}
    print(fold_dfs)

    #### GRAPH LOGOS ####
    freq_dfs = []        
    fc_counter = 0
    
    ## PLOT LOGOS AT EACH Log2FC Cutoff
    for k, v in fold_dfs.items():
        temp_df = v  # Select fold-change-filtered df for graphing
        print(f"\n#### N FC{float(k):0.3f}x Motifs: {temp_df.shape[0]}", temp_df)

        ## TABULATE POSITION-WISE FREQUENCY FOR SPECIFIC FOLD-CHANGE FILTER
        array_dict = {}
        rows = []
        nucleotide_order = ["A", "C", "G", "T"]

        for i in range(pam_length):
            array_dict[i] = Counter([x[i] for x in list(temp_df.Kmer)])

        for i in range(pam_length):
            v = array_dict[i]
            row_sum = sum(v.values())
            if row_sum == 0:
                rows.append(np.zeros(4))
            else:
                row = [v[n] / row_sum for n in nucleotide_order]
                rows.append(row)

        freq_df = pd.DataFrame(rows, columns=nucleotide_order)
        freq_dfs.append(freq_df)

        ## PLOT WEB LOGO
        # Format graph output
        fc_str = str(f"{fold_change_magnitudes[fc_counter]:0.4f}").replace('.', 'Pt')
        sample = test_condition_pam_file.split("/")[-1].split(".")[0].\
                         replace(' ', '_').replace('-', '_')
        ctrl = neg_ctrl_pam_file.split("/")[-1].split(".")[0].replace(' ', '_').replace('-', '_')
        file_base = f"Weblogo_{sample}_vs_{ctrl}_FC{fc_str}"
            
        if out_dir:
            file_base = f"{out_dir}{file_base.replace('.fa', '')}"

        # Save Fasta of PAMs at specific fold-change
        print(f"## SAVING FOLD-CHANGE FASTA: {file_base}.fa")
        write_df_to_fasta(df=temp_df, title=f"{file_base}")
        
        # Setup formatting for weblogo graph
        if not weblogo_x_axis_ticks:
            weblogo_x_axis_ticks = ', '.join([str(-i) for i in range(1, pam_length + 1)][::-1])
        if not weblogo_graph_title:
            weblogo_graph_title = f"{fc_str}XFC\n(n = {temp_df.shape[0]})"
        print(f"{fold_change_magnitudes[fc_counter]}")
            
        logo_from_fasta(fasta = f"{file_base}.fa", graph_title = weblogo_graph_title, 
                        pam_len=pam_length, to_save=True, out_name=file_base)
        print(f"## SAVING BIT-WEBLOGO: {file_base}.svg")
        
        fc_counter+=1

    return fold_dfs, freq_dfs, count_df

def nonparametric_ci_of_mean(data_array, n_iterations = 1000, alpha = 0.1, two_tail = False, mean = True):
    # Perform boostrapping
    boot_samples = []
    for _ in range(n_iterations):
        # RANDOM SAMPLE WITH REPLACEMENT
        boot_sample = np.random.choice(data_array, size=len(data_array), replace=True)
        boot_samples.append(boot_sample)

    # MEAN CI
    if mean:
        if two_tail:
            # Calculate the lower and upper bounds of the confidence interval
            boot_means = np.mean(boot_samples, axis=1)
            lower_bound = np.percentile(boot_means, alpha/2)
            upper_bound = np.percentile(boot_means, 100-alpha/2)
            return (lower_bound, upper_bound)
        else:
            boot_means = np.mean(boot_samples, axis=1)
            lower_bound = np.percentile(boot_means, alpha)
            return (lower_bound)
    # MIN CI
    else:
        if two_tail:
            # Calculate the lower and upper bounds of the confidence interval
            boot_mins = np.min(boot_samples, axis=1)
            lower_bound = np.percentile(boot_mins, alpha/2)
            upper_bound = np.percentile(boot_mins, 100-alpha/2)
            return (lower_bound, upper_bound)
        else:
            boot_mins = np.min(boot_samples, axis=1)
            lower_bound = np.percentile(boot_mins, alpha)
            return (lower_bound)
        

In [5]:
#### GET DATA AND SET VARIABLES ####
## DIRECTORIES
pwd = "/groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/Finalized_Code_for_Manuscript/"
results_dir = "/groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/GeoR1W1_Amy/Results/"
data_dir = "/groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/GeoR1W1_Amy/FASTQS/"
out_dir = f"{pwd}Logo_Output/"

## READS
fastqs = [x for x in glob.glob(f"{data_dir}/*/*.gz")]
r1_suffix = "_L001_R1_001.fastq.gz"
r2_suffix = "_L001_R2_001.fastq.gz"
base_files = [x.replace(r1_suffix, "") for x in fastqs if r1_suffix in x]
base_ids = [x.split("/")[-1].replace(r1_suffix, "") for x in fastqs if r1_suffix in x]

## DETAILS FOR REGEX: Define flanking regions for 4N Variable region
left_flank = "TTTTT"
right_flank = "CGAGT"
len_variable = 4

# DEFINE CONTROLS: Match targeting samples with original substrate library ("Amy1Libonly_S1")
control_dict = {"Amy5TR1W1_S5":"Amy1Libonly_S1", 
                "Amy4TWT_S4":"Amy1Libonly_S1",
                "Amy2NTWT_S2":"Amy1Libonly_S1", # Check for technical bias
                "Amy3NTR1W1_S3":"Amy1Libonly_S1", # Check for technical bias
               }


In [7]:
#### MERGE READS AND REGEX-EXTRACT PAMS ####

## MERGE FASTQS
merged_reads = [x for x in glob.glob(f"{data_dir}/*/*_merged_reads.fastq.gz", recursive=True)]
merged_reads = []

for base in base_files:
    temp_merged = fastp_merge_reads(r1=f"{base}{r1_suffix}", r2=f"{base}{r2_suffix}", base = base)
    merged_reads.append(temp_merged)

## GREP MERGED READS
seq_depth_stats = {}
match_files = []

for merged in merged_reads: 
    base = merged.split("/")[-1].replace("_merged_reads.fastq.gz", "")
    out = results_dir + base + "_matches.txt"
    
    ## GREP
    out_file_name, sequencing_depth, number_of_pattern_matches = grep_fastq(fasta=merged, 
                                                                            left_pat=left_flank, 
                                                                            right_pat=right_flank, 
                                                                            length_variable_region=len_variable, 
                                                                            out_file_name=out
                                                                           )
    seq_depth_stats[base] = [sequencing_depth, number_of_pattern_matches]
    match_files.append(out_file_name)
    
seq_depth_stats

zcat /groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/GeoR1W1_Amy/FASTQS/Amy2NTWT_L1_ds.fe4be173d93a4e6fb36ccf7e52a45c50/Amy2NTWT_S2_merged_reads.fastq.gz| grep -Eo 'TTTTT[A|T|G|C]{4}CGAGT' | sed -e 's/CGAGT$//' | sed -e 's/^TTTTT//' > /groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/GeoR1W1_Amy/Results/Amy2NTWT_S2_matches.txt
zcat /groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/GeoR1W1_Amy/FASTQS/Amy1Libonly_L1_ds.0f546033cc7844a19aea680acba40d6b/Amy1Libonly_S1_merged_reads.fastq.gz| grep -Eo 'TTTTT[A|T|G|C]{4}CGAGT' | sed -e 's/CGAGT$//' | sed -e 's/^TTTTT//' > /groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/GeoR1W1_Amy/Results/Amy1Libonly_S1_matches.txt
zcat /groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/GeoR1W1_Amy/FASTQS/Amy4TWT_L1_ds.3667e07b117a4ebc9638504bed6d04db/Amy4TWT_S4_merged_reads.fastq.gz| grep -Eo 'TTTTT[A|T|G|C]{4}CGAGT' | sed -e 's/CGAGT$//' | sed -e 's/^TTTTT//' > /groups/doudna/proj

{'Amy2NTWT_S2': [5928042.0, 2941051],
 'Amy1Libonly_S1': [6319458.0, 3134270],
 'Amy4TWT_S4': [8353252.0, 4144283],
 'Amy3NTR1W1_S3': [5970588.0, 2960647],
 'Amy5TR1W1_S5': [7402122.0, 3671516]}

In [ ]:
#### GET FOLD CHANGES AND EXPLORATORY LOGOS ####
"""
- Calculate Log2Fold Change against construct library ("Amy1Libonly_S1")
- Generate Log2 Fold-change dataframes for later processing
"""

fold_dfs = {}
freq_dfs = {} 
count_dfs = {}
cis = {}

## FOR EACH TARGETING SAMPLE, CALCULATE FOLDCHANGE RELATIVE TO SUBSTRATE LIBRARY
for base in control_dict.keys(): 
    test_condition_pam_file = f"{results_dir}{base}_matches.txt"
    neg_ctrl = control_dict[base]
    neg_ctrl_pam_file = f"{results_dir}{neg_ctrl}_matches.txt"
    
    ## GET PAM FOLDCHANGES
    fold_df, freq_df, count_df = pam_fold_change_logos(neg_ctrl_pam_file, 
                                                       test_condition_pam_file,
                                                       fold_change_magnitudes=[1, 2], # log2 fold-changes
                                                       depletion=True,
                                                       use_pseudo_counts=True,
                                                       length_variable_region=len_variable,
                                                       neg_ctrl_sequencing_depth=seq_depth_stats[neg_ctrl][0],
                                                       test_condition_sequencing_depth=seq_depth_stats[base][0],
                                                       reverse_complement=False,
                                                       bc_trim_n_left=False,
                                                       bc_trim_n_right=False,
                                                       weblogo_graph_title=False,
                                                       weblogo_x_axis_ticks=False,
                                                       out_dir=out_dir
                                                      )
    fold_dfs[base] = fold_df
    freq_dfs[base] = freq_df
    count_dfs[base] = count_df

{'1':     Kmer  Count_NTC  Count_Test  Log2_Fold_Change
56  CTAA   0.001783    0.000782         -1.188849
58  CGAA   0.001263    0.000600         -1.073967, '2': Empty DataFrame
Columns: [Kmer, Count_NTC, Count_Test, Log2_Fold_Change]
Index: []}

#### N FC1.000x Motifs: 2     Kmer  Count_NTC  Count_Test  Log2_Fold_Change
56  CTAA   0.001783    0.000782         -1.188849
58  CGAA   0.001263    0.000600         -1.073967
## SAVING FOLD-CHANGE FASTA: /groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/Finalized_Code_for_Manuscript/Logo_Output/Weblogo_Amy5TR1W1_S5_matches_vs_Amy1Libonly_S1_matches_FC1Pt0000.fa
1
## SAVING BIT-WEBLOGO: /groups/doudna/projects/mtrinidad_projects/Geo_Cas9_Indel_HDR_KC/Finalized_Code_for_Manuscript/Logo_Output/Weblogo_Amy5TR1W1_S5_matches_vs_Amy1Libonly_S1_matches_FC1Pt0000.svg

#### N FC2.000x Motifs: 0 Empty DataFrame
Columns: [Kmer, Count_NTC, Count_Test, Log2_Fold_Change]
Index: []
## SAVING FOLD-CHANGE FASTA: /groups/doudna/projects/mtrinidad

In [ ]:
#### GET CONDFIDENCE INTERVALS OF NEGATIVE CONTROLS ####
"""
- Use the non-targeting, negative controls (Amy3NTR1W1_S3, Amy2NTWT_S2) to establish
  a threshold for significantly depelted PAMs that are outside the 99.999999% CI for noise
- Establish CI's with bootstrapping
"""

ci_filter_dict = {"Amy5TR1W1_S5":"Amy3NTR1W1_S3",
                  "Amy4TWT_S4":"Amy2NTWT_S2"}
alpha = 10**-6
cis = {}

for sample,base in ci_filter_dict.items():
    print(base)
    ci = nonparametric_ci_of_mean(count_dfs[base].Log2_Fold_Change, 
                                  n_iterations = 100000, 
                                  alpha = alpha,
                                  mean = False
                                 )
    cis[sample] = ci
    
cis

In [ ]:
#### GENERATE LOGOS FOR STATISTICALLY SIGNIFICANT DEPELTION ####
"""
- Build logos from PAMs with L2FC Depletion > 99.999999% CI for Noise in respective non-targeting condition
"""

for base in ['Amy5TR1W1_S5', 'Amy4TWT_S4']: 
    test_condition_pam_file = f"{results_dir}{base}_matches.txt"
    neg_ctrl = control_dict[base]
    neg_ctrl_pam_file = f"{results_dir}{neg_ctrl}_matches.txt"
    significant_fold_change = -1*cis[base]
    
    ## GET PAM FOLDCHANGES
    fold_df, freq_df, count_df = pam_fold_change_logos(neg_ctrl_pam_file, 
                                                       test_condition_pam_file,
                                                       fold_change_magnitudes=[significant_fold_change], #log2fC
                                                       depletion=True,
                                                       use_pseudo_counts=True,
                                                       length_variable_region=len_variable,
                                                       neg_ctrl_sequencing_depth=seq_depth_stats[neg_ctrl][0],
                                                       test_condition_sequencing_depth=seq_depth_stats[base][0],
                                                       reverse_complement=False,
                                                       bc_trim_n_left=False,
                                                       bc_trim_n_right=False,
                                                       weblogo_graph_title=False,
                                                       weblogo_x_axis_ticks=False,
                                                       out_dir=out_dir
                                                      )
    fold_dfs[base] = fold_df
    freq_dfs[base] = freq_df
    count_dfs[base] = count_df

In [ ]:
#### GET CONSENSUS MOTIFS ####
# FASTAS WITH SIGNIFICANTLY DEPLETED PAMs
sig_fasta_R1W1 = f"{pwd}Logo_Output/Weblogo_Amy5TR1W1_S5_matches_vs_Amy1Libonly_S1_matches_FC0Pt0929.fa"
sig_fasta_WT = f"{pwd}Logo_Output/Weblogo_Amy4TWT_S4_matches_vs_Amy1Libonly_S1_matches_FC0Pt1003.fa"

# iGeo
r1w1_reads = get_fasta_seqs(sig_fasta_R1W1)
m_r1w1 = motifs.create(r1w1_reads)
print("R1W1 Consenus:", m_r1w1.degenerate_consensus)

# WT GEO
wt_reads = get_fasta_seqs(sig_fasta_WT)
m_wt = motifs.create(wt_reads)
print("WT Consenus:", m_wt.degenerate_consensus)